# Start working on LLM explanation

In [1]:
import dill
import shap
import pandas as pd
import numpy as np
import os

/home/syeda/.pyenv/versions/scam_job_detector/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "../models/final_model.dill"
preprocessor_path = "../models/preprocessor.dill"
data_path = "../raw_data/data_cleaned.csv"

model_path, preprocessor_path, data_path

('../models/final_model.dill',
 '../models/preprocessor.dill',
 '../raw_data/data_cleaned.csv')

In [3]:
# load the data
df = pd.read_csv(data_path)
df.head(1)

,has_company_logo,employment_type,industry,function,fraudulent,job_description,country
0,1,Other,NaN,Marketing,0,market intern food weve create groundbreaking ...,US


In [4]:
print(df.columns.tolist())

['has_company_logo', 'employment_type', 'industry', 'function', 'fraudulent', 'job_description', 'country']


In [5]:
# load the model and preprocessor
with open(model_path, "rb") as f:
    model = dill.load(f)

with open(preprocessor_path, "rb") as f:
    preprocessor = dill.load(f)

print("Model:", model)
print("Preprocessor loaded:", type(preprocessor))

Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=11, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=275, n_jobs=-1,
              num_parallel_tree=None, ...)
Preprocessor loaded: <class 'sklearn.compose._column_transformer.ColumnTransformer'>


# Streamlit Output stimulation

In [6]:
sample = df.sample(1, random_state=42)
sample

,has_company_logo,employment_type,industry,function,fraudulent,job_description,country
4708,1,Full-time,Apparel & Fashion,Information Technology,0,python engineer stylect dynamic startup help h...,GB


In [7]:
# remove target if present
X_sample = sample.drop(columns=["fraudulent"], errors="ignore")

# sanity check
X_sample

,has_company_logo,employment_type,industry,function,job_description,country
4708,1,Full-time,Apparel & Fashion,Information Technology,python engineer stylect dynamic startup help h...,GB


In [8]:
X_transformed = preprocessor.transform(X_sample)

X_transformed.shape

(1, 381412)

In [9]:
prediction = model.predict(X_transformed)[0]
probability = model.predict_proba(X_transformed)[0][1]

label = "FAKE" if prediction == 1 else "REAL"

print(f"Prediction: {label}")
print(f"Fake probability: {probability:.3f}")


Prediction: REAL
Fake probability: 0.000


# SHAP 

In [10]:
type(model)

xgboost.sklearn.XGBClassifier

In [11]:
dir(model)

['_Booster',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__sklearn_is_fitted__',
 '__sklearn_tags__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_request_for_signature',
 '_can_use_inplace_predict',
 '_configure_fit',
 '_create_dmatrix',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_get_default_requests',
 '_get_doc_link',
 '_get_iteration_range',
 '_get_metadata_request',
 '_get_param_names',
 '_get_params_html',
 '_get_type',
 '_html_repr',
 '_load_model_attributes',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_set_evaluation_result',
 '_update_sklearn_tag

In [12]:
def predict_proba_fn(X):
    return model.predict_proba(X)[:, 1]

In [13]:
background_df = (
    df
    .drop(columns=["fraudulent"], errors="ignore")
    .sample(20, random_state=42)
)

X_background = preprocessor.transform(background_df)


In [14]:
X_background.shape

(20, 381412)

In [15]:
X_transformed.shape

(1, 381412)

In [16]:
explainer = shap.KernelExplainer(
    predict_proba_fn,
    X_background
)


In [17]:
shap_values = explainer.shap_values(X_transformed)

  0%|          | 0/1 [00:00<?, ?it/s]/home/syeda/.pyenv/versions/scam_job_detector/lib/python3.10/site-packages/xgboost/data.py:1281: UserWarning: Unknown data type: <class 'scipy.sparse._lil.lil_matrix'>, trying to convert it to csr_matrix
  warnings.warn(
100%|██████████| 1/1 [03:26<00:00, 206.92s/it]


In [18]:
shap_values.shape

(1, 381412)

In [19]:
feature_names = preprocessor.get_feature_names_out()
len(feature_names), shap_values.shape[1]


(381412, 381412)

In [20]:
shap_df = pd.DataFrame({
    "feature": feature_names,
    "shap_value": shap_values[0]
})

shap_df["abs_value"] = shap_df["shap_value"].abs()

top_features = shap_df.sort_values(
    "abs_value", ascending=False
).head(10)

In [21]:
top_features[["feature", "shap_value", "abs_value"]]

,feature,shap_value,abs_value
263,pipeline-2__has_company_logo_0,-0.007957,0.007957
83,pipeline-1__country_US,-0.005375,0.005375
108334,columntransformer__tfidfvectorizer__engineer,-0.004791,0.004791
352159,columntransformer__tfidfvectorizer__understand,-0.004705,0.004705
354727,columntransformer__tfidfvectorizer__urgent,-0.004293,0.004293
371119,columntransformer__tfidfvectorizer__work,-0.004232,0.004232
29865,columntransformer__tfidfvectorizer__base,-0.004045,0.004045
330369,columntransformer__tfidfvectorizer__systems,-0.003857,0.003857
138686,columntransformer__tfidfvectorizer__fun,-0.003818,0.003818
224610,columntransformer__tfidfvectorizer__offshore,-0.003738,0.003738


In [22]:
for _, row in top_features.iterrows():
    direction = "FAKE ↑" if row["shap_value"] > 0 else "REAL ↓"
    print(f"{row['feature']}: {direction} ({row['shap_value']:.4f})")


pipeline-2__has_company_logo_0: REAL ↓ (-0.0080)
pipeline-1__country_US: REAL ↓ (-0.0054)
columntransformer__tfidfvectorizer__engineer: REAL ↓ (-0.0048)
columntransformer__tfidfvectorizer__understand: REAL ↓ (-0.0047)
columntransformer__tfidfvectorizer__urgent: REAL ↓ (-0.0043)
columntransformer__tfidfvectorizer__work: REAL ↓ (-0.0042)
columntransformer__tfidfvectorizer__base: REAL ↓ (-0.0040)
columntransformer__tfidfvectorizer__systems: REAL ↓ (-0.0039)
columntransformer__tfidfvectorizer__fun: REAL ↓ (-0.0038)
columntransformer__tfidfvectorizer__offshore: REAL ↓ (-0.0037)


In [23]:
def clean_feature_name(feature: str) -> str:
    # TF-IDF word features
    if "tfidfvectorizer__" in feature:
        return f"word '{feature.split('__')[-1]}' in job description"

    # binary categorical features
    if "has_company_logo" in feature:
        return "company has a logo"

    if "country_" in feature:
        return f"job country is {feature.split('_')[-1]}"

    # fallback
    return feature


In [24]:
top_features["clean_feature"] = top_features["feature"].apply(clean_feature_name)

In [92]:
explanation_payload = {
    "model_prediction": {},
    "top_features": []
}

# model-level prediction
explanation_payload["model_prediction"] = {
    "label": "FAKE" if prediction == 1 else "REAL",
    "fake_probability": float(probability)
}

# feature-level explanations (SHAP)
for _, row in top_features.iterrows():
    explanation_payload["top_features"].append({
        "feature": row["clean_feature"],
        "shap_value": float(row["shap_value"]),
        "direction": "FAKE" if row["shap_value"] > 0 else "REAL"
    })


In [93]:
def probability_to_confidence(p):
    if p < 0.01:
        return "very confident"
    elif p < 0.1:
        return "confident"
    elif p < 0.3:
        return "somewhat confident"
    else:
        return "uncertain"
confidence = probability_to_confidence(
    explanation_payload["model_prediction"]["fake_probability"]
)

explanation_payload["model_prediction"]["confidence"] = confidence

# Remove raw probability so LLM cannot leak math
del explanation_payload["model_prediction"]["fake_probability"]


In [94]:
from pprint import pprint
pprint(explanation_payload)

{'model_prediction': {'confidence': 'very confident', 'label': 'REAL'},
 'top_features': [{'direction': 'REAL',
                   'feature': 'company has a logo',
                   'shap_value': -0.007957412213090482},
                  {'direction': 'REAL',
                   'feature': 'job country is US',
                   'shap_value': -0.00537509447735382},
                  {'direction': 'REAL',
                   'feature': "word 'engineer' in job description",
                   'shap_value': -0.004791453156631669},
                  {'direction': 'REAL',
                   'feature': "word 'understand' in job description",
                   'shap_value': -0.004704634724138865},
                  {'direction': 'REAL',
                   'feature': "word 'urgent' in job description",
                   'shap_value': -0.0042934836200071765},
                  {'direction': 'REAL',
                   'feature': "word 'work' in job description",
                   'shap_value':

In [95]:
from dotenv import load_dotenv
import os
import json

load_dotenv()

True

In [96]:
import google.generativeai as genai
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [97]:
model = genai.GenerativeModel("models/gemini-flash-lite-latest")
response = model.generate_content("Say hello in one sentence")
print(response.text)

Hello!


In [ ]:
system_prompt = """
You explain machine learning predictions to everyday users.

Style rules:
- Sound natural, calm, and human
- Do NOT sound technical or academic
- Do NOT mention numbers, percentages, or formulas
- Do NOT mention probabilities explicitly
- Use short, clear sentences

Content rules:
- Use ONLY the provided information
- Do NOT invent features
- Do NOT change the prediction
"""


user_prompt = f"""
You are given the output of a machine learning model.

Write EXACTLY 3 short sentences.

Sentence 1:
- Clearly say whether the job is REAL or FAKE
- Mention confidence using words only (for example: very confident, confident)

Sentence 2:
- Explain the top 3 reasons in plain English
- Refer only to the listed features

Sentence 3:
- Give a short, reassuring summary for the user

JSON:
{json.dumps(explanation_payload, indent=2)}
"""


In [100]:
response = model.generate_content(
    system_prompt + "\n\n" + user_prompt,
    generation_config={
        "temperature": 0.2,
        "max_output_tokens": 500
    }
)

print(response.text)


This job looks very real. The company having a logo is a strong sign. The job being in the US and the word 'engineer' in the description also point to it being real. You can feel good about this job listing.
